In [1]:
# ================================================================
# Python Script to Load Sample Data to MySQL
# ================================================================

# Data path: D:\Data Analyst\Final Projects\MySQL+Python+PowerBI\1\Raw Data

import pandas as pd
import mysql.connector
from mysql.connector import Error
import os

def connect_to_mysql():
    """Connect to your MySQL database"""
    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',        # Change this to your MySQL username
            password='root',    # Change this to your MySQL password
            database='ecommerce_project'
        )
        print("✅ Connected to MySQL successfully!")
        return connection
    except Error as e:
        print(f"❌ Error connecting to MySQL: {e}")
        return None


def load_sample_data():
    """Load sample data from CSV files to MySQL tables"""
    
    print("🚀 Starting Sample Data Load...")
    print("=" * 50)
    
    # Your data path
    data_path = r"D:\Data Analyst\Final Projects\MySQL+Python+PowerBI\1\Raw Data"
    print(f"📁 Looking for CSV files in: {data_path}")
    
    # Connect to database
    connection = connect_to_mysql()
    if not connection:
        return
    
    cursor = connection.cursor()
    
    # File mapping: CSV filename -> MySQL table name
    file_table_mapping = {
        'df_Customers.csv': 'customers',
        'df_Products.csv': 'products', 
        'df_Orders.csv': 'orders',
        'df_OrderItems.csv': 'order_items',
        'df_Payments.csv': 'payments'
    }
    
    for csv_file, table_name in file_table_mapping.items():
        try:
            print(f"\n📁 Loading {csv_file}...")
            
            # Full path to CSV file
            full_path = f"{data_path}\\{csv_file}"
            
            # Read CSV file 
            df = pd.read_csv(full_path)
            print(f"   📊 Loaded {len(df):,} rows from CSV")
            
            # Clean column names
            df.columns = df.columns.str.strip().str.lower()

            # 🧹 Clean any ' UTC' text from datetime columns
            df = df.replace(' UTC', '', regex=True)
            
            # 🧹 Replace blanks/NaN with None → NULL in MySQL
            df = df.where(pd.notnull(df), None)
            df = df.replace(r'^\s*$', None, regex=True)
            
            # 🧹 Remove duplicate rows based on the first column (usually the PK)
            pk_col = df.columns[0]   # assumes 1st column is primary key
            before = len(df)
            df = df.drop_duplicates(subset=[pk_col], keep='first')
            removed = before - len(df)
            if removed > 0:
                print(f"   ⚠️ Removed {removed:,} duplicate records based on '{pk_col}'")
            
            # Clear existing data in table
            cursor.execute(f"DELETE FROM {table_name}")
            
            # Insert data row by row
            rows_inserted = 0
            for index, row in df.iterrows():
                try:
                    placeholders = ', '.join(['%s'] * len(row))
                    columns = ', '.join(df.columns)
                    sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
                    cursor.execute(sql, tuple(row))
                    rows_inserted += 1

                    if rows_inserted % 1000 == 0:
                        print(f"   ⏳ Inserted {rows_inserted:,} rows...")

                except Error as e:
                    print(f"   ⚠️ Row {index} failed in {table_name}: {e}")
                    continue
            
            # Commit the changes
            connection.commit()
            print(f"   ✅ Successfully inserted {rows_inserted:,} rows into {table_name}")
            
        except Exception as e:
            print(f"   ❌ Error loading {csv_file}: {e}")
            continue
    
    # Show final summary
    print(f"\n🎉 Sample Data Loading Complete!")
    print("=" * 50)
    
    tables = ['customers', 'products', 'orders', 'order_items', 'payments']
    for table in tables:
        try:
            cursor.execute(f"SELECT COUNT(*) FROM {table}")
            count = cursor.fetchone()[0]
            print(f"📊 {table}: {count:,} records")
        except:
            print(f"❌ {table}: Error getting count")
    
    cursor.close()
    connection.close()
    print("\n✅ All done! Ready for analysis!")


def test_database_connection():
    """Quick test to make sure everything works"""
    print("🔍 Testing Database Connection...")
    
    connection = connect_to_mysql()
    if connection:
        cursor = connection.cursor()
        cursor.execute("SELECT COUNT(*) FROM customers LIMIT 5")
        result = cursor.fetchone()
        print(f"✅ Test successful! Found {result[0]} customers")
        cursor.close()
        connection.close()
        return True
    return False


if __name__ == "__main__":
    print("🎯 E-commerce Sample Data Loader")
    print()
    
    if test_database_connection():
        print("✅ Database connection works!")
        input("\nPress Enter to start loading sample data...")
        load_sample_data()
    else:
        print("❌ Please check your MySQL connection settings")
        print("Update the username and password in the connect_to_mysql() function")


🎯 E-commerce Sample Data Loader

🔍 Testing Database Connection...
✅ Connected to MySQL successfully!
✅ Test successful! Found 0 customers
✅ Database connection works!



Press Enter to start loading sample data... 


🚀 Starting Sample Data Load...
📁 Looking for CSV files in: D:\Data Analyst\Final Projects\MySQL+Python+PowerBI\1\Raw Data
✅ Connected to MySQL successfully!

📁 Loading df_Customers.csv...
   📊 Loaded 89,316 rows from CSV
   ⏳ Inserted 1,000 rows...
   ⏳ Inserted 2,000 rows...
   ⏳ Inserted 3,000 rows...
   ⏳ Inserted 4,000 rows...
   ⏳ Inserted 5,000 rows...
   ⏳ Inserted 6,000 rows...
   ⏳ Inserted 7,000 rows...
   ⏳ Inserted 8,000 rows...
   ⏳ Inserted 9,000 rows...
   ⏳ Inserted 10,000 rows...
   ⏳ Inserted 11,000 rows...
   ⏳ Inserted 12,000 rows...
   ⏳ Inserted 13,000 rows...
   ⏳ Inserted 14,000 rows...
   ⏳ Inserted 15,000 rows...
   ⏳ Inserted 16,000 rows...
   ⏳ Inserted 17,000 rows...
   ⏳ Inserted 18,000 rows...
   ⏳ Inserted 19,000 rows...
   ⏳ Inserted 20,000 rows...
   ⏳ Inserted 21,000 rows...
   ⏳ Inserted 22,000 rows...
   ⏳ Inserted 23,000 rows...
   ⏳ Inserted 24,000 rows...
   ⏳ Inserted 25,000 rows...
   ⏳ Inserted 26,000 rows...
   ⏳ Inserted 27,000 rows...
   ⏳ 